In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader

from format_data import read_dataset, aggregate_dataset, clean_dataset, sliding_window
from dataset import MoodDataset

In [16]:
dataset = pd.read_csv('data/dataset_mood_smartphone.csv')
dataset = read_dataset(dataset)
dataset = aggregate_dataset(dataset)

# Remove rows without mood
dataset = dataset[dataset['mood'].notnull()]

all_labels = list(dataset['mood'])

# Fill NaN cells with average column value 
dataset = dataset.fillna(dataset.mean())

# drop_columns = ['mood']

# dataset = dataset.drop(columns=drop_columns)

stored_ids = dataset['id']
stored_dates = dataset['date']

dataset = dataset.drop(['id', 'date'], axis=1)
stored_columns = dataset.columns

# # dataset = dataset.values #returns a numpy array
# ss = preprocessing.StandardScaler()
# dataset = ss.fit_transform(dataset)
# dataset = pd.DataFrame(dataset, columns=stored_columns)

dataset.insert(0, 'id', list(stored_ids), allow_duplicates=True)
dataset.insert(1, 'date', list(stored_dates), allow_duplicates=True)
dataset.insert(2, 'label', all_labels, allow_duplicates=True)

display(dataset)

samples = sliding_window(dataset)
print(samples)

/var/folders/jr/91tgzxq96b30vd94j032zrwc0000gn/T/ipykernel_5517/4145430136.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dataset = dataset.fillna(dataset.mean())


variable,id,date,label,activity,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,appCat.office,...,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,circumplex.arousal,circumplex.valence,mood,screen,sms
7,AS14.01,2014-02-26,6.250000,0.119123,1427.33935,2736.186754,1220.576757,99.101126,522.55564,450.342019,...,308.017969,154.974224,107.278922,45.531324,1.000000,-0.250000,0.750000,6.250000,6081.640869,2.000000
8,AS14.01,2014-02-27,6.333333,0.119123,1427.33935,2736.186754,1220.576757,99.101126,522.55564,450.342019,...,308.017969,154.974224,107.278922,45.531324,3.925676,0.000000,0.333333,6.333333,6081.640869,2.464953
26,AS14.01,2014-03-21,6.200000,0.134050,3139.21800,6280.890000,1007.456000,49.544000,522.55564,172.206000,...,915.445000,154.974224,598.754000,45.531324,6.000000,0.200000,0.200000,6.200000,17978.907000,2.464953
27,AS14.01,2014-03-22,6.400000,0.236880,731.42900,4962.918000,93.324000,21.076000,522.55564,450.342019,...,37.305000,154.974224,117.621000,45.531324,3.000000,0.600000,0.500000,6.400000,6142.161000,1.000000
28,AS14.01,2014-03-23,6.800000,0.142741,1286.24600,5237.319000,94.346000,43.403000,522.55564,450.342019,...,308.017969,154.974224,30.086000,30.386000,3.925676,0.200000,0.800000,6.800000,6773.832001,2.464953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968,AS14.33,2014-05-27,6.200000,0.012704,726.39100,2078.403000,124.694000,99.101126,522.55564,450.342019,...,308.017969,154.974224,56.173000,45.531324,1.000000,-0.600000,0.400000,6.200000,4089.879001,2.000000
1969,AS14.33,2014-05-28,8.200000,0.103301,2551.04600,1936.415000,614.706000,99.101126,522.55564,357.909000,...,308.017969,154.974224,30.666000,45.531324,10.000000,0.000000,1.200000,8.200000,14320.867998,1.000000
1970,AS14.33,2014-05-29,7.000000,0.169354,400.03400,435.105000,136.738000,99.101126,522.55564,450.342019,...,0.939000,154.974224,3.199000,45.531324,5.000000,-1.333333,1.000000,7.000000,3569.341000,1.000000
1971,AS14.33,2014-05-30,6.800000,0.192901,3044.03000,1670.842000,260.220000,99.101126,522.55564,450.342019,...,1052.648000,8.072000,232.825000,45.531324,4.000000,-0.800000,-0.400000,6.800000,9497.646999,2.464953


[([array([0.11912254032246264, 2437.0460000000003, 3151.8289999999997,
       88.392, 99.1011262135922, 522.5556402116404, 450.34201851851867,
       408.106, 181.591, 308.0179688995216, 154.97422352941174,
       107.27892216981134, 45.53132432432432, 4.0, 0.0, 0.0,
       6.333333333333333, 4902.626999616626, 1.0], dtype=object), array([0.1680684149503257, 1229.347, 4518.116, 2511.178, 99.1011262135922,
       522.5556402116404, 450.34201851851867, 103.699, 103.02799999999999,
       308.0179688995216, 154.97422352941174, 107.27892216981134,
       45.53132432432432, 4.0, 0.5, 0.5, 6.75, 10159.769001245493,
       2.4649532710280373], dtype=object), array([0.21948438425543756, 2604.5209999999997, 3933.081, 88.943,
       99.1011262135922, 522.5556402116404, 450.34201851851867,
       88.72200000000001, 212.364, 308.0179688995216, 154.97422352941174,
       107.27892216981134, 45.53132432432432, 15.0, 0.6, 0.8, 8.2,
       4512.4710013866425, 1.0], dtype=object), array([0.315631756850

In [17]:
# dataset.var(ddof=0)

In [18]:
X = []
y = []

for index, sample in enumerate(samples):
    X.append([])
    for day_features in sample[0]:
        X[index].append(np.array(day_features))
    X[index] = np.array(X[index])
    y.append(sample[1])    
    
X = np.array(X)
y = np.array(y)
print(np.shape(X))
print(np.shape(y))

(805, 5, 19)
(805,)


In [19]:
# Testing fake data to see if model makes sense

# X = [np.array([1]) for x in range(100)]
# X += [np.array([2]) for y in range(100)]
# X = np.array(X)

# y = [1 for x in range(100)]
# y += [2 for y in range(100)]
# y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

train_dataset = MoodDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=50, shuffle=True)
test_dataset = MoodDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=50, shuffle=True)

In [20]:
class LSTM(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, output_size) :
        super().__init__()
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_size)
        self.dropout = nn.Dropout(0.0)
        
    def forward(self, x, dropout):
        if dropout:
            x = self.dropout(x)
        else:
            pass
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [ ]:
num_epochs = 10000
learning_rate = 0.00001

embedding_dim = 19
hidden_size = 100
output_size = 1
lstm = LSTM(embedding_dim, hidden_size, output_size)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for step, (x, y) in enumerate(train_dataloader): 
        
        outputs = lstm(x, True)

        optimizer.zero_grad()

        # obtain the loss function
        loss = criterion(outputs, y.float())

        loss.backward()

        optimizer.step()
        
    if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
        
        correct = 0
        total = 0
        
        total_mse = 0
        
        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for step, (x, y) in enumerate(test_dataloader): 

                # calculate outputs by running images through the network
                outputs = lstm(x, False)
                total_mse += criterion(outputs, y.float())/50
                # the class with the highest energy is what we choose as prediction
                predicted = outputs

                total += 50
                correct += (torch.round(predicted.squeeze()) == torch.round(y)).sum().item()
    
                
        print(f'MSE of the network on the test data: {total_mse}')
        print(f'Accuracy of the network on the test data: {100 * correct // total}%')

/opt/miniconda3/envs/dmt/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/miniconda3/envs/dmt/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([39])) that is different to the input size (torch.Size([39, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/miniconda3/envs/dmt/lib/python3.9/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have th

Epoch: 0, loss: 46.91684
MSE of the network on the test data: 5.7696614265441895
Accuracy of the network on the test data: 0%
Epoch: 100, loss: 38.75405
MSE of the network on the test data: 4.785552024841309
Accuracy of the network on the test data: 0%
Epoch: 200, loss: 35.25175
MSE of the network on the test data: 3.949268341064453
Accuracy of the network on the test data: 0%
Epoch: 300, loss: 26.25552
MSE of the network on the test data: 3.0907750129699707
Accuracy of the network on the test data: 0%
Epoch: 400, loss: 19.71327
MSE of the network on the test data: 2.459512710571289
Accuracy of the network on the test data: 0%
Epoch: 500, loss: 17.93458
MSE of the network on the test data: 1.880707025527954
Accuracy of the network on the test data: 0%
Epoch: 600, loss: 11.12393
MSE of the network on the test data: 1.3688232898712158
Accuracy of the network on the test data: 0%
Epoch: 700, loss: 7.47815
MSE of the network on the test data: 0.9160736799240112
Accuracy of the network on t

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for step, (x, y) in enumerate(test_dataloader): 
        
        # calculate outputs by running images through the network
        outputs = lstm(x, False)
        # the class with the highest energy is what we choose as prediction
        predicted = torch.argmax(outputs, dim=1)
        y = torch.argmax(y, dim=1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

print(f'Accuracy of the network on the test data: {100 * correct // total}%')